In [4]:
"""
Author:       Erfan Azad (erfan@dartmouth.edu)
Date:         01 February 2017
Description:  Simulation for SARSA-Lambda algorithm for a
              random walk in n states  
"""

'\nAuthor:       Erfan Azad (erfan@dartmouth.edu)\nDate:         01 February 2017\nDescription:  Simulation for SARSA-Lambda algorithm for a\n              random walk in n states  \n'

In [89]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [90]:
def choose_eps_greedy_action(state_actions, currentState, eps):
    '''
    Recieves a list of states and their actions, 
    and selects an action in a eps-greedy fashion
    for the current state using the given epsilon.
    
    Args:
        state_actions: nd-array of states and their action values.
        
        currentState: The state that the action will be selected for.
        
        eps: Epsilon variable used in epsilon-greedy action selection.
        
    Returns:
        The index of the action-value corresponding to an action.
        e.g. 1,2,.. (Note: 0 is the index for the state itself)
    '''
    assert (currentState > 0 and currentState < state_actions.shape[0]-1)
    if (np.random.random() > eps):
        action = np.argmax(state_actions[currentState,1:]) + 1 # column of the Q corresponding to the greedy action in the state_actions
    else:
        action = np.random.choice([1,2])  # ACTION 1 or ACTION 2
    return action

In [91]:
def calculate_reward(st, numStates):
    '''
    Calculates the reward for the 1-D random walk problem.
    
    Args:
        st: The state that the agent is heading to.
        
        numStates: Total number of states including the terminal
                states.
    
    Returns:
        The reward of
        -1 for state 0,
        +1 for the right most state (index = numStates - 1)
        0 for any other state in between.
    '''
    reward = None
    if (st != 0 and st != numStates-1):
        reward = 0
    elif (st == 0):
        reward = -1
    else:
        reward = 1
    return reward

In [92]:
def takeStep(state_actions, currentState, numStates, epsilon):
    '''
    Takes a step in the episode.
    
    Args:
        state_actions: a (numStates x 3) size array representing
                    each state s and its action-values:  Q(s,a)
        
        currentState: current state that we are taking the step from.
        
        epsilon: epsilon variable used in the epsilon-greedy action selection.
        
    Returns: 
        Action taken, observed reward, and the next state in form of [a, r, s_next]
    '''
    action = choose_eps_greedy_action(state_actions, currentState, epsilon) # Choose and action in current state 
    next_st = (currentState + 1 if action==2 else currentState -1)          # Observe next state
    reward = calculate_reward(next_st, numStates)                           # Observe the reward
    
    return [action, reward, next_st]

In [93]:
def buildEpisode_SARSA(state_actions, numStates, epsilon):
    '''
    Builds an episode for SARSA_Lambda for
    a 1-D random walk problem.
    
    Args:
        state_actions: a (numStates x 3) size array representing
                    each state s and its action-values:  Q(s,a)
        
        numStates: Number of states including the two
                terminal states.
                
        epsilon: epsilon that is used in the eps-greedy
                action selection method. determins the 
                extent of taking explorative actions.
                e.g. epsilon=0.1 --> 10% of the time 
    
    returns:
        And array containing state, action, reward history
        e.g. [[s0,a0,r0], [s1,a1,r1], ...]
    '''
    assert (numStates > 2)
    state_action_reward_history = []
    st = numStates/2          # Start from the middle state
    reward = None             # Initializing the reward
    while (st > 0 and st < numStates-1):
        action, reward, next_st = takeStep(state_actions, st, numStates, epsilon)
        state_action_reward_history.append([st, action, reward])  # Record [currentState, action, reward]
        st = next_st
    return state_action_reward_history

In [94]:
def SARSA_Lambda_offline(lambda_factor, gamma, alpha, epsilon, numStates, numEpisodes):
    """
    Runs the SARSA-Lambda learning algorithm for 
    the random walk problem. It used the offline
    version such that it will update all the
    action-values, Q(s,a), after each episode.
    
    Args:
        lambda_factor:      decay factor ("lambda" is a keyword in python!)
        gamma:       discount factor
        alpha:       learning rate
        numStates:   number of states (including the terminal states)
        numEpisodes: number of times/ episodes to repeat the learning
    
    Returns:
        The learned values of each state: V(s)
    """
    state_actions = np.vstack((np.arange(numStates), np.zeros((2,numStates)))).T # e.g. row0 ==> [S0,Q1,Q2] for numState rows
    e_trace = np.zeros((numStates, 2))
    deltaQ = np.zeros((numStates, 2))
    for i in range(numEpisodes):
        history = buildEpisode_SARSA(state_actions, numStates, epsilon)
        T = len(history)
        err = 0
        for t in range(0,T):
            #err = rt + gamma*Q(st+1, ai) - Q(st,ai)
            reward = history[t][2]
            Q = state_actions[history[t][0], history[t][1]]
            try:
                Q_next = state_actions[history[t+1][0], history[t+1][1]]
            except IndexError:
                Q_next = 0
            err = reward + gamma*Q_next - Q
            e_trace[history[t][0],history[t][1]-1] += 1
            
            deltaQ = (1-alpha)*deltaQ + alpha*err*e_trace
            e_trace = gamma*lambda_factor*e_trace
            e_trace[e_trace < 0.0001] = 0
        state_actions[:,1:] += deltaQ
    return np.round(state_actions, 2)

In [95]:
def SARSA_Lambda_online(lambda_factor, gamma, alpha, epsilon, numStates, numEpisodes):
    """
    Runs the SARSA-Lambda learning algorithm for 
    the random walk problem. It used the online
    version such that it will update the
    action-values, Q(s,a), at each step of the episode.
    
    Args:
        lambda_factor:      decay factor ("lambda" is a keyword in python!)
        gamma:       discount factor
        alpha:       learning rate
        numStates:   number of states (including the terminal states)
        numEpisodes: number of times/ episodes to repeat the learning
    
    Returns:
        The learned values of each state: V(s)
    """
    state_actions = np.vstack((np.arange(numStates), np.zeros((2,numStates)))).T # e.g. row0 ==> [S0,Q1,Q2] for numState rows
    e_trace = np.zeros((numStates, 2))
    for i in range(numEpisodes):
        currentState = numStates /2 # Start from the middle
        while (currentState >0  and currentState < numStates-1):
            action, reward, nextState = takeStep(state_actions, currentState, numStates, epsilon) # Take action, observe reward and the nextState
            Q = state_actions[currentState, action]
            try:
                nextAction =  choose_eps_greedy_action(state_actions, nextState, epsilon)         # Choose nextAction from nextState
                Q_next = state_actions[nextState, nextAction]
            except AssertionError:
                Q_next = 0 # nextState is a Terminal State
            err = reward + gamma*Q_next - Q
            e_trace[currentState, action - 1] += 1
            # For all s, a update the Q(s,a) and e_trace(s,a)
            state_actions[:,1:] += alpha*err*e_trace
            e_trace = gamma*lambda_factor*e_trace
            
            # Update currentState
            currentState = nextState
    return np.round(state_actions, 2)

In [72]:
t1 = time.time()
result = SARSA_Lambda_offline(lambda_factor=0.5,gamma=0.9, alpha=0.1, epsilon=0.2, numStates=15, numEpisodes=10000)
t2 = time.time()
print(result)
print("Finished in {} seconds.".format(t2-t1))

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00  -1.00000000e-01  -0.00000000e+00]
 [  2.00000000e+00  -5.00000000e-02  -0.00000000e+00]
 [  3.00000000e+00  -2.00000000e-02  -0.00000000e+00]
 [  4.00000000e+00  -1.00000000e-02   1.50000000e-01]
 [  5.00000000e+00   8.00000000e-02   3.70000000e-01]
 [  6.00000000e+00   3.00000000e-01   4.20000000e-01]
 [  7.00000000e+00   3.70000000e-01   4.90000000e-01]
 [  8.00000000e+00   4.30000000e-01   5.60000000e-01]
 [  9.00000000e+00   4.70000000e-01   6.30000000e-01]
 [  1.00000000e+01   5.50000000e-01   7.00000000e-01]
 [  1.10000000e+01   6.20000000e-01   7.90000000e-01]
 [  1.20000000e+01   7.00000000e-01   9.00000000e-01]
 [  1.30000000e+01   8.00000000e-01   1.00000000e+00]
 [  1.40000000e+01   0.00000000e+00   0.00000000e+00]]
Finished in 24.8786411285 seconds.


In [97]:
t1 = time.time()
result = SARSA_Lambda_online(lambda_factor=0.5,gamma=0.9, alpha=0.1, epsilon=0.2, numStates=45, numEpisodes=10000)
t2 = time.time()
print(result)
print("Finished in {} seconds.".format(t2-t1))

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00  -1.00000000e-01   0.00000000e+00]
 [  2.00000000e+00  -4.00000000e-02   0.00000000e+00]
 [  3.00000000e+00  -2.00000000e-02   0.00000000e+00]
 [  4.00000000e+00  -1.00000000e-02   0.00000000e+00]
 [  5.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  6.00000000e+00  -0.00000000e+00  -0.00000000e+00]
 [  7.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  8.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  9.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  1.00000000e+01  -0.00000000e+00  -0.00000000e+00]
 [  1.10000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.20000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.30000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.40000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.50000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.60000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.70000000e+01  -0.00000000e+00   0.00000000e+00]
 [  1.80000000e+01   0.00000